<a href="https://colab.research.google.com/github/v370r/projects/blob/master/housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_file = "/content/drive/MyDrive/kaggle/home-data-for-ml-course/train.csv"
data = pd.read_csv(test_file,index_col="Id") #making index column as start
data.describe()

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
  data["LotArea"].dtype

In [ ]:
%%time
data1 = pd.DataFrame()
for x in data.columns:
    data1[x] = data[x].astype("object")
data1.dtypes

In [ ]:
data.describe()

In [ ]:
round(data["LotArea"].mean(),2)

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(axis=0)

In [ ]:
data.isnull().sum()

In [63]:
Y = data.SalePrice

In [ ]:
feature_names = ["LotArea","YearBuilt","1stFlrSF","2ndFlrSF","FullBath","BedroomAbvGr","TotRmsAbvGrd"]
x = data[feature_names]
x.describe(include= "all")

In [ ]:
x.head()

# Model Fit

`Decision tree regressor`

In [66]:
from sklearn.tree import DecisionTreeRegressor
iowa_model = DecisionTreeRegressor(random_state=2)

In [ ]:
iowa_model.fit(x,Y)

In [ ]:
z = x[:1]
z["LotArea"] = 8000
z["YearBuilt"] =2000
z["TotRmsAbvGrd"]= 6.5
z

In [ ]:

predictions= iowa_model.predict(z)
predictions

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,Y,random_state=1,test_size=0.6) #train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
model = DecisionTreeRegressor(random_state=1)
model.fit(train_x,train_y)


In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
predict_y= model.predict(test_x)
print(predict_y[:5])

In [ ]:
from sklearn.metrics import r2_score  #r2 or coeefiecent of determination
r2_score(test_y,predict_y)

In [75]:
from sklearn.ensemble import RandomForestRegressor
? RandomForestRegressor

In [ ]:
"""
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  #(src,destination)
    os.symlink("../input/home-data-for-ml-course/test.csv",
  """

In [ ]:
"""
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]
print(models)
"""

In [ ]:
%%time
from sklearn.metrics import mean_absolute_error
test_mae = abs(predict_y-test_y).mean()
test_mae

In [79]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
def get_mae(max_leaf_node,train_x,test_x,train_y,test_y):
  model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_node,random_state=1)
  model.fit(train_x,train_y)
  predictions = model.predict(test_x)
  mae  = mean_absolute_error(test_y,predictions)
  return(mae)

In [ ]:
%%time
ret = float("inf")
for max_leaf_nodes in range(2,1000,10):
  my_mae = get_mae(max_leaf_nodes,train_x,test_x,train_y,test_y)
  ret = min(ret,my_mae)
print(ret)

In [81]:
from sklearn.ensemble import RandomForestRegressor
def get_mae2(max_leaf_node,train_x,test_x,train_y,test_y):
  model = RandomForestRegressor(max_leaf_nodes=max_leaf_node,random_state=1)
  model.fit(train_x,train_y)
  predictions = model.predict(test_x)
  mae  = mean_absolute_error(test_y,predictions)
  return(mae)


In [ ]:
%%time
ret = float("inf")
for max_leaf_nodes in range(2,1000,10):
  my_mae = get_mae2(max_leaf_nodes,train_x,test_x,train_y,test_y)
  ret = min(ret,my_mae)
print(ret)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state =1)
rf_model.fit(train_x,train_y)
rf_predictions = rf_model.predict(test_x)
rf_predictions
r2_score(test_y,rf_predictions)


In [ ]:
rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
rf.fit(train_x, train_y)


In [ ]:
!pip install rfpimp

In [ ]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances
def r2(rf, train_x, train_y):
    return r2_score(train_y, rf.predict(train_x))
perm_imp_rfpimp = permutation_importances(rf, train_x, train_y, r2)

In [ ]:
perm_imp_rfpimp

In [ ]:
test_y.shape

In [ ]:
test_y.index.sort_values(ascending=True)[100:120]

In [ ]:
predict_y.shape

In [113]:
output = pd.DataFrame({"Id":test_y.index,"cost":test_y,'SalePrice rf': rf_predictions,"sales trees":predict_y})
output.to_csv('/content/drive/MyDrive/kaggle/home-data-for-ml-course/submission.csv', index=False)